In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datos_perfumes_paco.csv")

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df.insert(0, 'tienda', "paco")

In [5]:
def convertir_a_minusculas(valor):
    if isinstance(valor, str):
        return valor.lower()
    return valor

df = df.map(convertir_a_minusculas)

In [6]:
df['categoria'] = df['categoria'].str.replace('perfumes de hombre', 'hombre')
df['categoria'] = df['categoria'].str.replace('perfumes de mujer', 'mujer')

In [7]:
def mapear_tipos_y_limpiar(df):    

    mapping = {
        'eau de  parfum'         : 'eau de parfum',
        'eau parfumee'           : 'eau de parfum',
        'eau parfumeé'           : 'eau de parfum',
        'extrait de parfum'      : 'parfum',
        'essence de parfum'      : 'parfum',
        'eau detoilette'         : 'eau de toilette',
        'eau fraîche'            : 'eau de cologne',
        'para hombre'            : '',
        'para mujer'             : '',
        'para  mujer'            : '',
        'unisex'                 : '',
        'intense'                : '',
        'extreme'                : '',
        'légère'                 : '',
        'cologne spray'          : '',
        'el ritual perfumado -'  : '',
        'y vaporizador de viaje' : ''
    }

    for name, replacement in mapping.items():
        df['tipo'] = df['tipo'].str.replace(name, replacement, regex=False)

    df['tipo'] = df['tipo'].str.strip()
    
    return df

mapear_tipos_y_limpiar(df)

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,paco,hombre,paco rabanne,1 million,eau de toilette,80.50,38%,80.50,5,4
1,paco,hombre,dior,sauvage,eau de parfum,77.94,no tiene,sin descuento,4,17
2,paco,hombre,paco rabanne,invictus,eau de toilette,79.00,38%,79.00,4,58
3,paco,hombre,armani,acqua di giò hombre,eau de toilette,70.00,50%,70.00,4,10
4,paco,hombre,loewe,esencia loewe,eau de toilette,97.00,35%,97.00,4,2
...,...,...,...,...,...,...,...,...,...,...
1563,paco,mujer,perris monte carlo,musk extreme,eau de parfum,145.00,no tiene,sin descuento,0,0
1564,paco,mujer,jimmy boyd,wonderful india,eau de toilette,62.00,15%,62.00,0,0
1565,paco,mujer,montale paris,velvet fantasy,eau de parfum,129.95,no tiene,sin descuento,0,0
1566,paco,mujer,loewe,aura floral,eau de parfum,130.00,35%,130.00,4,5


In [8]:
df.loc[df['tipo'] == 'cologne', 'tipo'] = 'eau de cologne'

In [9]:
def clean_prices(df, columns):
    
    for col in columns:
        
        df[col] = df[col].str.replace('[^0-9,]', '', regex=True) 
        df[col] = df[col].str.replace(',', '.', regex=True)
        df[col] = df[col].replace('', '0', regex=True)
        df[col] = df[col].astype(float)  

columns = ['descuento', 'precio_con_desc']

clean_prices(df, columns)

In [10]:
df['precio_con_desc'] = (df['precio_sin_desc'] * (1 - df['descuento'] / 100)).round(2)

In [11]:
columns = ['valoracion', 'num_valoraciones']

for col in columns:
    df[col] = df[col].replace(['sin valoracion', 'sin valoraciones'], np.nan)

In [12]:
df = df.drop_duplicates()

In [13]:
df['tipo'].value_counts()

tipo
eau de parfum                    792
eau de toilette                  714
parfum                            34
eau de cologne                    27
agua de tratamiento perfumada      1
Name: count, dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568 entries, 0 to 1567
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tienda            1568 non-null   object 
 1   categoria         1568 non-null   object 
 2   marca             1568 non-null   object 
 3   nombre            1568 non-null   object 
 4   tipo              1568 non-null   object 
 5   precio_sin_desc   1568 non-null   float64
 6   descuento         1568 non-null   float64
 7   precio_con_desc   1568 non-null   float64
 8   valoracion        1568 non-null   int64  
 9   num_valoraciones  1568 non-null   int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 122.6+ KB


In [15]:
lista_tipos = ['eau de parfum', 'eau de toilette', 'parfum', 'eau de cologne']
df = df[df["tipo"].isin(lista_tipos)].reset_index(drop=True)

In [16]:
df.to_csv('datos_perfumes_paco_filtrados.csv', index=False)